<a href="https://colab.research.google.com/github/Jaskaran7d5/BCS-secy-recruitment/blob/main/Sorrytuning2_JaskaranSingh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
#importing libraries
import numpy as np
import pandas as pd
%matplotlib inline
#to use as command line calls #using inline graphs will come next to code

import matplotlib.pyplot as plt #for graphs
import os #for operating system dependent fucntionality
from keras import layers #for building layers of neural net
from keras.models import Model
from keras.models import load_model
from keras import callbacks #for training logs, saving to disk periodically
import cv2 #OpenCV(Open Source computer vision lib), containg CV algos
import string


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CaptchaDataset/dataset2/mapping.csv')

In [ ]:
df.head()

,file_name,captcha
0,1.png,rP677
1,2.png,8396J
2,3.png,90p4g
3,4.png,08KSN
4,5.png,2CETY


In [ ]:
data = df.to_numpy()
imgshape=(50,200,1)
nchar = 36
character = 'abcdefghijklmnopqrstuvwxyz1234567890'

In [ ]:
data.shape

(82329, 2)

In [ ]:
n=82329

In [ ]:
X = np.zeros((n,50,200,1)) #1070*50*200 array with all entries 0
y = np.zeros((6,n,nchar)) #5*1070*36(5 letters in captcha) with all entries 0

for i in range(0,n):
    if i==24 or i==85:
      continue
  #i represents index no. of image in directory
  #pic contains the file name of the particular image to be preprocessed at a time

    img = cv2.imread(os.path.join("/content/drive/MyDrive/CaptchaDataset/dataset1/images", data[i,0]), cv2.IMREAD_GRAYSCALE) #Read image in grayscale format
    pic_target = data[i,0][:-4]#this drops the .png extension from file name and contains only the captcha for training
    print(f"i is {i} while pic is {data[i,0]}")
    try:
      img = img / 255.0 #scales the image between 0 and 1
    except:
      continue
    img = np.reshape(img, (50, 200, 1)) #reshapes image to width 200 , height 50 ,channel 1

    target=np.zeros((6,nchar)) #creates an array of size 5*36 with all entries 0

    for j, k in enumerate(data[i,1]):
      #j iterates from 0 to 4(5 letters in captcha)
      #k denotes the letter in captcha which is to be scanned
         print(f"char is {k}")
         index = character.find(k) #index stores the position of letter k of captcha in the character string
         target[j, index] = 1 #replaces 0 with 1 in the target array at the position of the letter in captcha

    X[i] = img #stores all the images
    y[:,i] = target #stores all the info about the letters in captcha of all images

Streaming output truncated to the last 5000 lines.
i is 77329 while pic is 77330.png
i is 77330 while pic is 77331.png
i is 77331 while pic is 77332.png
i is 77332 while pic is 77333.png
i is 77333 while pic is 77334.png
i is 77334 while pic is 77335.png
i is 77335 while pic is 77336.png
i is 77336 while pic is 77337.png
i is 77337 while pic is 77338.png
i is 77338 while pic is 77339.png
i is 77339 while pic is 77340.png
i is 77340 while pic is 77341.png
i is 77341 while pic is 77342.png
i is 77342 while pic is 77343.png
i is 77343 while pic is 77344.png
i is 77344 while pic is 77345.png
i is 77345 while pic is 77346.png
i is 77346 while pic is 77347.png
i is 77347 while pic is 77348.png
i is 77348 while pic is 77349.png
i is 77349 while pic is 77350.png
i is 77350 while pic is 77351.png
i is 77351 while pic is 77352.png
i is 77352 while pic is 77353.png
i is 77353 while pic is 77354.png
i is 77354 while pic is 77355.png
i is 77355 while pic is 77356.png
i is 77356 while pic is 77357.p

In [ ]:
def createmodel():
    img = layers.Input(shape=imgshape) # Get image as an input of size 50,200,1
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img) #50*200
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 25*100
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 13*50
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3) #to improve the stability of model
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 7*25

    flat = layers.Flatten()(mp3) #convert the layer into 1-D

    outs = []
    for _ in range(5): #for 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1) #drops 0.5 fraction of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res) #result of layers

    # Compile model and return it
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model



In [ ]:
model=createmodel();
model.summary();

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 200, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 50, 200, 16)          160       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 25, 100, 16)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 25, 100, 32)          4640      ['max_pooling2d[0][0]']   

In [ ]:
X_train, y_train = X[:80000], y[:, :80000]
X_test, y_test = X[80000:], y[:, 80000:]

code is crashing after this line. Not able to train model due to insufficient RAM

In [ ]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=3, validation_split=0.2)